In [11]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import os
from glob import glob
import time
import zipfile
import plotly.graph_objects as go
import plotly.express as px

In [12]:
anio=str(input('desde que anio: '))
link_consulta='https://comex.indec.gob.ar/?_ga=2.1947448.887743671.1661390997-1687177890.1630124319#/database'

In [13]:
def inicio_driver(link:str):
    service = Service(ChromeDriverManager().install())
    carpeta_descarga=os.getcwd().replace('src','downloads')
    prefs = {'download.default_directory' : carpeta_descarga,
        "directory_upgrade": True}
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_experimental_option("prefs", prefs)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(link)
    driver.maximize_window()
    return driver

def every_downloads_chrome(driver):
    '''Para ver cuando terminan las descargas'''
    if not driver.current_url.startswith("chrome://downloads"):
        driver.get("chrome://downloads/")
    return driver.execute_script("""
        var items = document.querySelector('downloads-manager')
            .shadowRoot.getElementById('downloadsList').items;
        if (items.every(e => e.state === "COMPLETE"))
            return items.map(e => e.fileUrl || e.file_url);
        """)

In [14]:
driver = inicio_driver(link_consulta)
def scrap_database(driver, frecuencia = "Mensual"):
    '''Frecuencia: Mensual o Anual'''
    frecuencia = frecuencia.capitalize()

    lista_botones=driver.find_elements(By.CLASS_NAME, "form-control")

    #A partir de acá, quiero  cambiar el anio hacia delante

    select_comercio=Select(lista_botones[0])
    select_comercio.select_by_value('exports')

    select_frecuencia=Select(lista_botones[2])
    select_frecuencia.select_by_visible_text(frecuencia)

    #Lo pongo despues para hacer el loop

    select_anio=Select(lista_botones[1])


    opciones=[]
    for option in select_anio.options: 
        opciones.append(option.get_attribute('value'))
    opciones.pop(0)
    # opciones=filter(lambda x: int(x)>=int(anio),opciones)
    opciones=list(filter(lambda x: x>=anio,opciones))

    for opcion in opciones:
        select_anio.select_by_visible_text(opcion)
        descarga=driver.find_element(By.CLASS_NAME, "btn-outline-primary")
        descarga.click()
        time.sleep(1)
        
    paths = WebDriverWait(driver, 300, 1).until(every_downloads_chrome)
    driver.quit()
    
    return opciones

opciones = scrap_database(driver,frecuencia="mensual")
scrap_database(driver, frecuencia="anual")



In [15]:
columnas_anuales=['Año', 'NCM', 'Pdes', 'Pnet(kg)', 'FOB(u$s)']

def concatenacion_dfs_anuales(opciones=opciones):
    '''Extrae y concatena las dfs'''
    dfs={}
    for idx, opcion in enumerate(opciones):
        with zipfile.ZipFile(f'../downloads/exports_{opcion}_Y.zip', 'r') as zip_ref:
            zip_ref.extractall('../downloads')
        try: df=pd.read_csv(f'../downloads/expona{opcion[2:]}.csv', sep=';', encoding='latin-1',decimal=',', dtype={'NCM':str, 'pdes':str})
        except: df=pd.read_csv(f'../downloads/expot{opcion[2:]}.csv', sep=';', encoding='latin-1',decimal=',', dtype={'NCM':str, 'pdes':str})
        df.rename(columns={col: col.lower() for col in columnas_anuales}, inplace=True)
        df.ncm=df.ncm.apply(lambda x: x.strip())
        dfs[str(2022-idx)] = df
    return pd.concat([dfs[key] for key in dfs.keys()])

df_anual=concatenacion_dfs_anuales()

In [16]:
columnas=['Año', 'Mes', 'NCM', 'Pdes', 'Pnet(kg)', 'FOB(u$s)', 'CIF(u$s)']
def concatenacion_dfs_mensuales(opciones=opciones):
    '''Extrae y concatena las dfs'''
    dfs={}
    for idx, opcion in enumerate(opciones):
        with zipfile.ZipFile(f'../downloads/exports_{opcion}_M.zip', 'r') as zip_ref:
            zip_ref.extractall('../downloads')
        try: df=pd.read_csv(f'../downloads/exponm{opcion[2:]}.csv', sep=';', encoding='latin-1',decimal=',', dtype={'NCM':str, 'pdes':str})
        except: df=pd.read_csv(f'../downloads/expom{opcion[2:]}.csv', sep=';', encoding='latin-1',decimal=',', dtype={'NCM':str, 'pdes':str})
        df.rename(columns={col: col.lower() for col in columnas}, inplace=True)
        df.ncm=df.ncm.apply(lambda x: x.strip())
        dfs[str(2022-idx)] = df
    return pd.concat([dfs[key] for key in dfs.keys()])

df_completa=concatenacion_dfs_mensuales()

In [17]:
def limpieza_y_correcion_dfs(df):
    df['pnet(kg)']=df['pnet(kg)'].astype(str).apply(lambda x: x.replace(',','.'))
    df['fob(u$s)']=df['fob(u$s)'].astype(str).apply(lambda x: x.replace(',','.'))
    df.loc[df['pnet(kg)'].str.contains("s"), 'pnet(kg)'] = "0"
    df.loc[df['fob(u$s)'].str.contains("s"), 'fob(u$s)'] = "0"
    df['pnet(kg)']=df['pnet(kg)'].astype(float)
    df['fob(u$s)']=df['fob(u$s)'].astype(float)
    df.pdes=df.pdes.astype(str)
    df.año=df.año.astype(str)
    return df

df_completa = limpieza_y_correcion_dfs(df_completa)
df_completa.mes=df_completa.mes.astype(str)
df_anual = limpieza_y_correcion_dfs(df_anual)

In [18]:
partida = df_anual.copy()
partida["partida"] = partida.ncm.apply(lambda x: x[:4])
partida.groupby(["año","partida"],as_index=False).sum()
# "{}"partida[(partida.partida == "4811")&(partida.año == "2018")]["fob(u$s)"].sum()


C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_10804\2645471985.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  partida.groupby(["año","partida"],as_index=False).sum()


,año,partida,pnet(kg),fob(u$s)
0,2019,0101,1.301600e+06,2.820670e+07
1,2019,0102,1.316300e+05,1.072409e+06
2,2019,0103,0.000000e+00,0.000000e+00
3,2019,0104,3.579000e+03,4.653628e+04
4,2019,0105,0.000000e+00,0.000000e+00
...,...,...,...,...
5145,2023,9702,0.000000e+00,0.000000e+00
5146,2023,9703,4.092300e+03,9.166562e+04
5147,2023,9705,0.000000e+00,0.000000e+00
5148,2023,9998,1.992130e+08,2.474655e+08


In [19]:
def plot_df_secreto(df = df_anual):
    df_secreto = df[df.ncm == "99999999"].copy()
    df_sin_secreto = df[(df.ncm != "99999999") & (df.año >= "2018")].copy()
    df_sin_secreto = df_sin_secreto.groupby("año",as_index=False).sum()
    prop_secreto = df_sin_secreto.merge(df_secreto[["año","fob(u$s)"]],how="left", on = "año")
    prop_secreto["proporcion"] = prop_secreto["fob(u$s)_y"] / prop_secreto["fob(u$s)_x"]*100
    fig = go.Figure()
    fig.add_trace(go.Bar(x = df_secreto.año, y = df_secreto["fob(u$s)"]/1000000, name = "Secreto"))
    fig.add_trace(go.Bar(x = df_sin_secreto.año, y = df_sin_secreto["fob(u$s)"]/1000000, name = "FOB sin secreto"))
    fig.update_yaxes(tickformat = ",")
    fig.update_layout(template = None, separators = ",.", title_text = "Secreto estadístico a nivel producto en millones de USD")
    return prop_secreto, fig
plot_df_secreto()[0]

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_10804\838485921.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_sin_secreto = df_sin_secreto.groupby("año",as_index=False).sum()


,año,pnet(kg),fob(u$s)_x,fob(u$s)_y,proporcion
0,2019,1.246168e+11,6.174265e+10,3.372675e+09,5.462473
1,2020,1.144252e+11,5.240502e+10,2.478800e+09,4.730080
2,2021,1.270206e+11,7.390435e+10,4.029968e+09,5.452951
3,2022,1.235363e+11,8.417510e+10,4.270615e+09,5.073489
4,2023,1.184904e+10,9.042552e+09,1.067297e+09,11.803048


In [20]:
#lista de paises

paises=pd.read_csv('../data/CE_PAIS.csv', sep=';', dtype={'CCOD_ASOC':str})
paises['cod_final']=paises.CCOD_ASOC.fillna(paises.CCOD_PAIS)
dic_paises=dict(zip(paises.CCOD_PAIS,paises.CDESCRI))
paises['descri_final'] = paises.cod_final.apply(lambda x: dic_paises[x])
dic_cod_paises=dict(zip(paises.CCOD_PAIS,paises.cod_final))
asociados=paises[~paises.CCOD_ASOC.isna()] #para ver cod con asociados
dic_paises_asociados=dict(zip(paises.CCOD_PAIS,paises.descri_final))
paises=paises[['cod_final','descri_final']]
#Paises

def inserta_paises(df):
    df.insert(4,'pais_descri',df.pdes.apply(lambda x: dic_paises[x]))
    df.insert(5,'cod_final',df.pdes.apply(lambda x: dic_cod_paises[x]))
    df.insert(6,'descri_final',df.pdes.apply(lambda x: dic_paises_asociados[x]))
    df = df.fillna(0)
    df = df.drop('pdes',axis=1)
    df = df.drop('pais_descri', axis = 1)
    return df

df_completa = inserta_paises(df_completa)
df_anual = inserta_paises(df_anual)

In [21]:
#Para ver el ultimo mes disponible
dic_meses={
    'Enero':1,
    'Febrero':2,
    'Marzo':3,
    'Abril':4,
    'Mayo':5,
    'Junio':6,
    'Julio':7,
    'Agosto':8,
    'Septiembre':9,
    'Octubre':10,
    'Noviembre':11,
    'Diciembre':12
}
dic_meses={v:k for k,v in dic_meses.items()}
def ultimo_mes(df = df_completa):
    df.año = df.año.astype(int)
    df.mes = df.mes.astype(int)
    df.sort_values(["año","mes"],ascending=True)
    return dic_meses[df.mes.iloc[-1]]

ultimo_mes_disponible = ultimo_mes(df_completa)
ultimo_anio_disponible = int(df_completa.sort_values(['año','mes'],ascending=False).reset_index(drop=True)
         .año.iloc[0])

ultimo_mes_disponible


'Diciembre'

In [22]:
descripciones_df = pd.read_csv('../data/CE_ENMIENDA.csv',sep=';',dtype={'ncm':str,'cnro_enmienda':int})
def put_descripciones_ncm(df):
    return df.merge(descripciones_df, on = 'ncm',how = 'left')
df_anual = put_descripciones_ncm(df_anual)
df_completa = put_descripciones_ncm(df_completa)

In [23]:
def acorta_descri(descri, largo = 45):
    if len(descri)>=largo:
        return descri[:largo]+'...'
    else:
        return descri
    

def ncm_fob_anual(anio_desde,df=df_anual, ncm='23040010'):
    ncm=str(ncm)
    anio_desde=str(anio_desde)
    df = df[df['año'] >= anio_desde].reset_index(drop = True)
    df=df[df.ncm==ncm].reset_index(drop=True)
    descri = acorta_descri(df.ncm_descri.iloc[0], largo = 60)
    df=df.groupby('año',as_index=False).sum()
    # titles = get_lista_paises(dfs)
             
    ncm_fob=go.Figure()
    # for i in range(len(dfs)):
    x=df['año']
    ncm_fob.add_trace(go.Bar(x=x, y=df["fob(u$s)"]/1000000, name="FOB Aceite de soja", textposition='inside',
    # text=dfs[i]["fob_licuado"]/1000, texttemplate = '%{text:.2f}'
    ))

    # ncm_fob.update_xaxes(showgrid=True, title_text="Mes", title_standoff=7)
    ncm_fob.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_fob.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_fob.update_layout(barmode='relative',separators=",.", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    xaxis = dict(tickmode = "linear"),
    template = 'none',
    title_text=f"FOB exportado de: <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
    legend = dict(yanchor="top",y=1, xanchor="left", x=1, orientation="v"))
    note = f'Fuente: @MartinBasualdo0 en base a INDEC. Datos del {ultimo_anio_disponible} hasta {ultimo_mes_disponible}'
    ncm_fob.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.5, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_fob

ncm_fob_anual(anio_desde=2007, ncm="27112100")


C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_10804\3768948586.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [24]:
def ncm_fob_anual_pais(ncm = "27112100"):
    prueba_df = df_anual.copy()
    prueba_df = prueba_df[prueba_df.ncm == ncm].reset_index(drop=True)
    descri = acorta_descri(prueba_df.ncm_descri.iloc[0], largo = 60)
    prueba_df = prueba_df.pivot_table(values="fob(u$s)", index="año", columns="descri_final")
    fig = go.Figure()
    for pais in prueba_df.columns:
        fig.add_trace(go.Bar(name = pais,x = prueba_df.index, y = prueba_df[pais]/1000000))
    fig.update_yaxes(tickformat = ",")
    fig.update_layout(template = None, separators = ".,", barmode="stack", 
        height=600, width=1000, 
        title=f"FOB exportado de: <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
        legend = dict(yanchor="bottom", xanchor="left", orientation="h", y = .9))
    note = f'Fuente: MartinBasualdo0 en base a INDEC'
    fig.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.26, yref='paper', y=-0.18,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,
    )
    return fig

ncm_fob_anual_pais()

In [25]:
def ncm_fob_mensual(anio_desde,df=df_completa, ncm='12019000'):
    ncm=str(ncm)
    # anio_desde=str(anio_desde)
    df = df[df['año'] >= anio_desde].reset_index(drop = True)

    df=df.sort_values(['año','mes'],ascending=True).reset_index(drop=True)
    año_final=df.año.iloc[-1]
    mes_final=df.mes.iloc[-1]
    hasta=str(int(mes_final)+1)+'/'+'01'+'/'+str(año_final)
    desde=str(df.mes.iloc[0])+'/'+'01'+'/'+str(df.año.iloc[0])

    df=df[df.ncm==ncm].reset_index(drop=True)
    descri = acorta_descri(df.ncm_descri.iloc[0], largo = 60)
    df=df.groupby(['año','mes'],as_index=False).sum()
    df.mes=df.mes.astype(int)
    df=df.sort_values(['año','mes'],ascending=True)
    df.mes=df.mes.astype(str)
    df['fecha']=df['mes'].astype(str)+'/'+df['año'].astype(str)
        
    monthDates = pd.DataFrame({
    'fecha': pd.date_range(start=desde, end=hasta, freq='M').strftime('%m-%Y')
    })
    
    ncm_cif=px.bar(df, x=df.fecha,y=df["fob(u$s)"]/1000000,
    title=f"FOB exportado de: <br>\"{descri}\"<br><sup>En millones de dólares</sup>",
    category_orders={'fecha':monthDates.fecha}, )

    ncm_cif.update_xaxes(type='category',title_text="",categoryarray=monthDates.fecha)
    ncm_cif.update_yaxes(title_text="Millones USD", tickformat=',')
    ncm_cif.update_layout(uniformtext_minsize=10, uniformtext_mode='hide')

    ncm_cif.update_layout(barmode='relative',separators=",.", 
                                            #  margin ={'t': 0},
    height=600, width=1000, 
    # xaxis = dict(tickmode = "linear"),
    template = 'none',
    # title_text=f"CIF importado de :<br>\"Poroto de soja\"<br><sup>En millones de dólares</sup>",
    legend = dict(yanchor="top",y=1.07, xanchor="left", x=0.0, orientation="h"))
    note = f'Fuente: MartinBasualdo0 en base a INDEC'
    ncm_cif.add_annotation(showarrow=False, text=note,font=dict(size=12), xref='paper', x=0.26, yref='paper', y=-0.18,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)
    return ncm_cif
    
ncm_fob_mensual(2019,ncm='10019900')


C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_10804\2465467411.py:14: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [26]:
df_completa.to_csv('../output/expo_desagregado_ncm_pais.csv')
df_anual.to_csv('../output/expo_anual.csv')

In [27]:
expo_anual = df_anual[df_anual.año == "2022"]["fob(u$s)"].sum()
# {":.0f"}.format(expo_anual)
"{:.2f}".format(expo_anual)

'88445718838.32'

In [28]:
subpartida = df_anual.copy()
subpartida["subpartida"] = subpartida.ncm.apply(lambda x: x[:4])
subpartida = subpartida.groupby(["año","subpartida"],as_index=False).sum()

C:\Users\mbasualdo\AppData\Local\Temp\ipykernel_10804\3412920298.py:3: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [29]:
for i in glob("../downloads/*", recursive = True): os.remove(i)